# Colossus demo: Cosmology

Welcome to the Colossus cosmology tutorial.

In [ ]:
from __future__ import print_function 
import matplotlib.pyplot as plt
%matplotlib inline

## Setting, changing, and adjusting cosmologies

Let's begin by importing the colossus cosmology module. If this call results in an error, please see the chapter "Installation" in the documentation.

In [ ]:
from colossus.cosmology import cosmology

Let's set a cosmology, e.g. the most recent Planck cosmology. This function returns a cosmology object:

In [ ]:
cosmo = cosmology.setCosmology('planck15')

Colossus comes with more than 20 hard-coded cosmologies, a list of which can be printed with this command:. An easy way to see what the properties of a cosmology are is to print it:

In [ ]:
for k in cosmology.cosmologies.keys():
    print(k)

If we want to retrieve details about a given cosmology, we can just print the cosmology object:

In [ ]:
print(cosmo)

In Colossus, the cosmology is set globally. This means that we can obtain the current cosmology object from anywhere in the code:

In [ ]:
cosmo = cosmology.getCurrent()
print(cosmo.name)

However, cosmologies are objects, meaning that we can keep multiple cosmologies around. This can be useful when switching back and forth between different cosmologies, for example:

In [ ]:
cosmo2 = cosmology.setCosmology('WMAP9')
cosmology.setCurrent(cosmo2)
print(cosmology.getCurrent().name)
cosmology.setCurrent(cosmo)
print(cosmology.getCurrent().name)

Of course, you can also set a user-defined cosmology. In this case, arguments to the constructor of the Cosmology object are passed as a dictionary. Note that therre are many more possible arguments than shown here, as detailed in the documentation.

In [ ]:
my_cosmo = {'flat': True, 'H0': 72.0, 'Om0': 0.25, 'Ob0': 0.043, 'sigma8': 0.8, 'ns': 0.97}
cosmo = cosmology.setCosmology('my_cosmo', my_cosmo)
print(cosmo)

By default, relativistic species (neutrinos and photons) are included, meaning that they contribute to the energy density of the universe and are subtracted from the remaining components. Thus, when setting the flat cosmology above with Om0=0.25 and relspecies=True, the density of dark energy is not 0.75 but rather 0.75 minus the density of relativistic species. In practice, the relativistic contribution is small except at very high redshifts and makes a negligible contribution to most calculations.

It is possible to change parameters of a cosmology after it has been created, but we need to alert Colossus of the change because it needs to clear the cache:

In [ ]:
cosmo.Om0 = 0.27
cosmo.checkForChangedCosmology()
print(cosmo)

## Standard calculations: Densities, distances, times

Let's perform some simple calculations such as cosmological densities, distances, and times. As before, we begin by setting a particular cosmology:

In [ ]:
cosmo = cosmology.setCosmology('planck15')

Using the object returned by this function (or cosmology.getCurrent()), we can evaluate standard cosmological quantities. For example, let's print the age of the universe today. The cosmology functions always take redshift rather than scale factor as an argument:

In [ ]:
cosmo.age(0.0)

Note that the returned age is in Gyr. The colossus cosmology functions adhere to the following unit system:

* Length: comoving Mpc/h
* Mass: Msun
* Wavenumber: comoving h/Mpc
* Time: Gyr
* Density: physical Msun * h^2 / kpc^3

Virtually all Colossus functions accept numpy arrays as input and return arrays of the same dimensions:

In [ ]:
import numpy as np
z = np.array([0.0, 1.0, 2.0, 3.0])
cosmo.angularDiameterDistance(z)

The colossus cosmology module works to pretty extreme redshifts, namely between z = -0.995 and 200 (though some functions may have stricter limits, for example z>=0). For example, let's plot the relative density contribution of the various components in the Planck cosmology:

In [ ]:
zp1 = 10**np.arange(-1.0, 2.0, 0.05)
z = zp1 - 1.0

O_b = cosmo.Om(z) * cosmo.Ob0 / cosmo.Om0
O_dm = cosmo.Om(z) * (1.0 - cosmo.Ob0 / cosmo.Om0)
O_L = cosmo.OL(z)
O_gamma = cosmo.Ogamma(z)
O_nu = cosmo.Onu(z)

plt.figure()
plt.loglog()
plt.xlabel('z+1')
plt.ylabel('Omega')
plt.plot(zp1, O_dm, '-', label = 'Dark matter')
plt.plot(zp1, O_b, '-', label = 'Baryons')
plt.plot(zp1, O_L, '-', label = 'Dark Energy')
plt.plot(zp1, O_gamma, '-', label = 'Photos')
plt.plot(zp1, O_nu, '-', label = 'Neutrinos')
plt.legend(loc = 4);

There is more to the functions mentioned above: they can also give us their own inverse, e.g. $z(t)$ instead of $t(z)$:

In [ ]:
z = np.array([0.0, 1.0, 2.0, 3.0])
t = cosmo.age(z)
z2 = cosmo.age(t, inverse = True)
print(z2)

The slight errors in the recovered redshifts give a hint as to how colossus works internally: the majority of functions rely on interpolation for performance. The interpolating splines can be inverted numerically. If, for some reason, you really need the exact calculation, you can set interpolation = False in the constructor, or at a later time.

Once again using interpolating splines, we can also evaluate the derivatives of many functions, e.g. $dt/dz$ or $d^2t/dz^2$:

In [ ]:
cosmo.age(z, derivative = 1)

In [ ]:
cosmo.age(z, derivative = 2)

We can even combine the inverse and derivative arguments to evaluate the derivative of the inverse, e.g. $dz/dt$:

In [ ]:
t = cosmo.age(z)
cosmo.age(t, inverse = True, derivative = 1)

A word of warning: while the documentation quotes a particular accuracy for each function, the accuracy of the derivatives and inverses is not guaranteed. There are many more functions related to densities, distances, and times. Please consult the documentation of the cosmology module for an exhaustive list.

## The distribution of matter: power spectrum, variance, correlation function

Colossus is geared toward calculations related to the evolution of structure in the universe and this focus is reflected in the cosmology module. We assume that the linear over- and underdensity field is given by a Gaussian random field which is unambiguously described by its power spectrum P(k). By default, colossus evaluates the matter power spectrum using the approximation of Eisenstein & Hu 1998, but the user can also pass a tabulated power spectrum (e.g., computed using CAMB). 

In [ ]:
k = 10**np.arange(-5,2,0.02)
Pk = cosmo.matterPowerSpectrum(k)

plt.figure()
plt.loglog()
plt.xlabel('k(Mpc/h)')
plt.ylabel('P(k)')
plt.plot(k, Pk, '-');

It is a little hard to discern features in the power spectrum with this many orders of magnitude on the plot. As the functions discussed above, the matterPowerSpectrum() function relies on interpolating splines and can return the derivative of the power spectrum, highlighting the BAO wiggles:

In [ ]:
Pk_deriv = cosmo.matterPowerSpectrum(k, derivative = True)

plt.figure()
plt.xscale('log')
plt.xlabel('k(h/Mpc)')
plt.ylabel('d log(P) / d log(k)')
plt.plot(k, Pk_deriv, '-');

A number of important quantities are integrals over the power spectrum, for example the matter-matter correlation function:

In [ ]:
R = 10**np.arange(0,2.4,0.005)
xi = cosmo.correlationFunction(R, 0.0)

plt.figure()
plt.loglog()
plt.xlabel('R(Mpc/h)')
plt.ylabel('abs(xi)')
plt.plot(R, np.abs(xi), '-');

Or the variance sigma(R):

In [ ]:
R = 10**np.arange(0,2.4,0.005)
sigma = cosmo.sigma(R, 0.0)

plt.figure()
plt.loglog()
plt.xlabel('R(Mpc/h)')
plt.ylabel('sigma(R)')
plt.plot(R, sigma, '-');

You may notice that the first time you execute functions the functions above, they can take up to a couple of seconds. That's because Colossus computes an interpolation table, saves it to disk, and loads it the next time the same cosmology is initiated. If, for some reason, this behavior is undesired, you can control it with the "storage" parameter to the constructor.